In [21]:
import pandas as pd
from google.colab import drive
from pandas import Series, DataFrame
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
from pandas import DataFrame
from pandas import concat
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [22]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset = dataset[what_to_use]
  return dataset

In [23]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 함수
def series_to_supervised(data, n_in=1, n_out=1, target = 'TARGET', dropnan=True):
    df = DataFrame(data)
    df.drop(target, axis = 1, inplace=True)
    df2 = DataFrame(data[target])
    cols, names = list(), list()
    n_vars = 1 if type(df) is list else df.shape[1]
    n_vars2 = 1 if type(df2) is list else df2.shape[1]
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df2.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [24]:
def seperator(dataset):
  
  # series_to_supervised 함수를 지나며 전처리를 하면 하나의 row에 이전 7일간의 데이터가 일렬로 들어가고(train), 여기에 target으로 미래 96일 발전량이 따라 붙는다.
  # 이 둘이 한 줄에 있으므로 적절한 지점에서 잘라서 train_X와 train_y로 사용한다.
  X = dataset.iloc[:, :n_obs]
  y = dataset.iloc[:, -future_window:]
  
  # validation을 해야하니까, train으로 준 데이터를 train/test로 자른다.
  # 사용가능한 연도가 3개년이고, 대충 train에 2년치, test에 1년치를 주었다 (7:3)
  # 42 = ultimate answer to life the universe and everything
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

  # 학습과 검증 데이터를 원하는 모양([1,4,2]) 이런 식으로 바꾸는 함수
  #### 처음 모양 (36490, 2016)
  # 2년치가 데이터(730일 * 하루에 48틱 = 36490개)로 있고, 이전 일주일 데이터 7*48 = 336인데 사용한 변수가 6개라면 336*6이다.
  # 일단 2016개의 데이터를 틱별로(30분 단위로) 잘라준다
  # 한 틱에 6개의 변수가 들어간다면 이를 n_features로 반영
  
  #### 수정할 모양 (36490, 336, 6)
  # 6개 특징이 30분 단위로 336개(일주일) 준비되어 있다.
  # 이런 데이터가 36490개 있어서 학습에 사용할 수 있다.

  # y 데이터는 1개 변수(발전량)만 나오고, 미래 2일 (2*48 = 96틱)이므로
  # 형태 ('총 데이터의 개수', '미래/과거의 길이', '사용한 변수의 개수')
  # == (36490, 96, 1) 으로 reshape 한다.

  train_X = X_train.values
  train_X = train_X.reshape(train_X.shape[0],-1,n_features)
  train_y = y_train.values
  train_y = train_y.reshape(train_y.shape[0],future_window)

  test_X = X_test.values
  test_X = test_X.reshape(test_X.shape[0],-1,n_features)
  test_y = y_test.values
  test_y = test_y.reshape(test_y.shape[0],future_window)

  return train_X, train_y, test_X, test_y

In [25]:
def data_maker(train_X, train_y, test_X, test_y):
  train_X = torch.tensor(np.array(train_X), dtype=torch.float)
  train_X = train_X.transpose(1,0)
  train_y = torch.tensor(np.array(train_y), dtype=torch.float)

  test_X = torch.tensor(np.array(test_X), dtype=torch.float)
  test_X = test_X.transpose(1,0)
  test_y = torch.tensor(np.array(test_y), dtype=torch.float)
  return train_X, train_y, test_X, test_y

In [26]:
############ 파라미터 #####################
# 하루의 틱
ticks = 48
# 예측에 사용할 일수
days = 7
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']
# 사용할 변수
what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

n_features = len(what_to_use) - 1
n_obs = n_days * n_features

# 한 번에 뭉테기로 투입할 자료의 양
batches = 100
# 몇 번이나 반복하여 학습할 것인다.
epoch = 200

In [29]:
def data_loader(dataset):

  ############ 파라미터 #####################
  # 하루의 틱
  ticks = 48
  # 예측에 사용할 일수
  days = 7
  n_days = ticks*days

  # 미래 예측할 일수
  future_days = 2
  future_window = ticks * future_days

  ### 모든변수
  # ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']
  # 사용할 변수
  what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

  n_features = len(what_to_use) - 1
  n_obs = n_days * n_features


  dataset = drop_clms(dataset)
  dataset = series_to_supervised(dataset, n_days, future_window, target='TARGET')
  train_X, train_y, test_X, test_y = seperator(dataset)
  train_X, train_y, test_X, test_y = data_maker(train_X, train_y, test_X, test_y)
  return train_X, train_y, test_X, test_y

,var1(t-336),var2(t-336),var3(t-336),var4(t-336),var5(t-336),var1(t-335),var2(t-335),var3(t-335),var4(t-335),var5(t-335),var1(t-334),var2(t-334),var3(t-334),var4(t-334),var5(t-334),var1(t-333),var2(t-333),var3(t-333),var4(t-333),var5(t-333),var1(t-332),var2(t-332),var3(t-332),var4(t-332),var5(t-332),var1(t-331),var2(t-331),var3(t-331),var4(t-331),var5(t-331),var1(t-330),var2(t-330),var3(t-330),var4(t-330),var5(t-330),var1(t-329),var2(t-329),var3(t-329),var4(t-329),var5(t-329),...,TARGET1(t+56),TARGET1(t+57),TARGET1(t+58),TARGET1(t+59),TARGET1(t+60),TARGET1(t+61),TARGET1(t+62),TARGET1(t+63),TARGET1(t+64),TARGET1(t+65),TARGET1(t+66),TARGET1(t+67),TARGET1(t+68),TARGET1(t+69),TARGET1(t+70),TARGET1(t+71),TARGET1(t+72),TARGET1(t+73),TARGET1(t+74),TARGET1(t+75),TARGET1(t+76),TARGET1(t+77),TARGET1(t+78),TARGET1(t+79),TARGET1(t+80),TARGET1(t+81),TARGET1(t+82),TARGET1(t+83),TARGET1(t+84),TARGET1(t+85),TARGET1(t+86),TARGET1(t+87),TARGET1(t+88),TARGET1(t+89),TARGET1(t+90),TARGET1(t+91),TARGET1(t+92),TARGET1(t+93),TARGET1(t+94),TARGET1(t+95)
336,0.0,0.0,0.0,69.08,-12.0,0.5,0.0,0.0,69.06,-12.0,1.0,0.0,0.0,71.78,-12.0,1.5,0.0,0.0,71.75,-12.0,2.0,0.0,0.0,75.20,-12.0,2.5,0.0,0.0,69.29,-11.0,3.0,0.0,0.0,72.56,-11.0,3.5,0.0,0.0,72.55,-11.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
337,0.5,0.0,0.0,69.06,-12.0,1.0,0.0,0.0,71.78,-12.0,1.5,0.0,0.0,71.75,-12.0,2.0,0.0,0.0,75.20,-12.0,2.5,0.0,0.0,69.29,-11.0,3.0,0.0,0.0,72.56,-11.0,3.5,0.0,0.0,72.55,-11.0,4.0,0.0,0.0,74.62,-11.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,1.0,0.0,0.0,71.78,-12.0,1.5,0.0,0.0,71.75,-12.0,2.0,0.0,0.0,75.20,-12.0,2.5,0.0,0.0,69.29,-11.0,3.0,0.0,0.0,72.56,-11.0,3.5,0.0,0.0,72.55,-11.0,4.0,0.0,0.0,74.62,-11.0,4.5,0.0,0.0,74.61,-11.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,1.5,0.0,0.0,71.75,-12.0,2.0,0.0,0.0,75.20,-12.0,2.5,0.0,0.0,69.29,-11.0,3.0,0.0,0.0,72.56,-11.0,3.5,0.0,0.0,72.55,-11.0,4.0,0.0,0.0,74.62,-11.0,4.5,0.0,0.0,74.61,-11.0,5.0,0.0,0.0,73.74,-11.0,...,0.0,0.0,0.0,0.0,0.000000,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
340,2.0,0.0,0.0,75.20,-12.0,2.5,0.0,0.0,69.29,-11.0,3.0,0.0,0.0,72.56,-11.0,3.5,0.0,0.0,72.55,-11.0,4.0,0.0,0.0,74.62,-11.0,4.5,0.0,0.0,74.61,-11.0,5.0,0.0,0.0,73.74,-11.0,5.5,0.0,0.0,73.73,-11.0,...,0.0,0.0,0.0,0.0,7.132752,15.015990,8.070917,12.575616,17.924748,20.270696,26.276539,30.687244,28.341125,29.467262,30.780751,44.106563,39.414376,33.596432,26.746057,19.144757,11.167897,3.847805,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52460,22.0,0.0,0.0,60.10,-4.0,22.5,0.0,0.0,60.08,-4.0,23.0,0.0,0.0,59.37,-4.0,23.5,0.0,0.0,59.35,-4.0,